# LSTM

## 由RNN引入

循环神经网络（RNN）能存储历史数据的信息，存储的历史数据将会影响将来的决策。

![带循环的递归神经网络](./LSTM-RNN.png)

在上图中，神经网络A接受某些输入$x_t$，并输出一个值$h_t$。循环允许信息从网路的下一步骤传递到下一个。

- 一个循环神经网络可以被认为是同一个网络的多个副本，每一个都传递一个消息后继者。

![](./LSTM-RNN-2.png)

RNN的缺点：

- RNN在实践中很难解决“长依赖”问题。

为了解决“长依赖”问题，LSTM应运而生。

## LSTM网络

LSTM（Long Short-Term Memory），长短时记忆神经网络，是一种特殊的RNN，能够学习长的依赖关系。

- LSTM是为了避免长依赖问题而精心设计的。记住较长的历史信息实际上是他们的默认行为，而不是他们努力学习的东西。

所有循环神经网络都具有神经网络的重复模块链的形式。在标准的RNN中，该重复模块将具有非常简单的结构，例如当个tanh层。

![](./LSTM-1.png)

LSTM也拥有这种链壮结构，但是重复模块则拥有不同的结构。与神经网络的简单一层相比，LSTM拥有四层，这四层以特殊的方式进行交互。

![](./LSTM-2.png)

一些符号定义：

![](./opts.png)

在上图中，每一行都带有一个向量，该向量从一个节点输出到其他节点的输入。 粉红色圆圈表示点向运算，如向量加法、点乘，而黄色框是学习神经网络层。 线的合并表示连接，而线的交叉表示其内容正在复制，副本将转到不同的位置

## LSTM 背后的核心理念

- LSTM的关键是**细胞状态**，表示细胞状态的这条线水平的穿过图的顶部。

细胞的状态类似于输送带，细胞的状态在整个链上运行，只有一些小的线性操作作用其上，信息很容易保持不变的流过整个链。

![](./LSTM-cell.png)

- LSTM确实具有删除或添加信息到细胞状态的能力，这个能力是由被称为**门(Gate)**的结构所赋予的。

门是一种可选地让信息通过的方式。它由一个Sigmoid神经网络和一点乘法运算组成。

![](./Gate.png)

Sigmoid神经网络层输出0和1之间的数字，这个数字描述每个组件有多少信息可以通过，0表示不通过任何信息，1表示全部通过。

### LSTM的三个门

- **LSTM有三个门，用于保护和控制细胞的状态**

#### 忘记门

- LSTM的第一步是决定我们要从细胞状态中丢弃什么信息。该决定由被称为“忘记门”的Sigmoid层实现，他查看$h_{t-1}$（前一个输出）和$x_t$（当前输入），并为单元格状态$C_{t-1}$(上一个状态)中的每个数字输出0和1之间的数字。1代表完全保留，而0代表彻底删除。

让我们回到语言模型的例子，试图根据以前的语料来预测下一个单词。 在这样的问题中，细胞状态可能包括当前主题的性别，从而决定使用正确的代词。 当我们看到一个新主题时，我们想要忘记旧主题的性别。

![](./Gate-1.png)

#### 输入门层

- 下一步是决定我们要在细胞状态中存储什么信息。这部分分为两步。

1. 首先，称为“输入门层”的Sigmoid层决定了我们将更新哪些值。
2. 接下来一个tanh层创建候选向量$\tilde{C}_t$会被加入到状态中。

在我们的语言模型的例子中，我们希望增加新的主语的性别到细胞状态中，来替代旧的需要忘记的主语。

![](./Gate-2.png)

- 现在是更新旧细胞状态的时间了，$C_{t-1}$更新为$C_t$。

![](./Gate-2-1.png)



#### 输出门层

最终，我们需要确定输出什么值。这个输出将会基于我们的细胞状态，但是也是一个过滤后的版本。首先，我们运行一个 sigmoid 层来确定细胞状态的哪个部分将输出出去。接着，我们把细胞状态通过 tanh 进行处理（得到一个在 -1 到 1 之间的值）并将它和 sigmoid 门的输出相乘，最终我们仅仅会输出我们确定输出的那部分。
在语言模型的例子中，因为他就看到了一个 代词，可能需要输出与一个 动词 相关的信息。例如，可能输出是否代词是单数还是负数，这样如果是动词的话，我们也知道动词需要进行的词形变化。

![](./Gate-3.png)